## Minimalna dvokubitna QBM

Kvantna Born mašina (QBM) je generativni kvantni model mašinskog učenja koji uči raspodele verovatnoća koristeći Bornovo pravilo. QBM-u se zadaje ciljana raspodela, a zatim model podešava parametre  kvantnog kola tako da raspodela merenja postane što sličnija zadatoj. Ova vežba prikazuje dvokubitni QBM sa jednim varijacionim parametrom.

![2-qubit-QBM](images/2-qubit-QBM.png)

Kvantno kolo modela sastoji se od:

1. `RY(θ)` rotacije prvog kubita.  
2. `CX(0→1)` operacije spletanja  kubita.  
3. Merenja u standardnoj bazi.  

Dobijeno stanje je:

$$
\cos\!\left(\tfrac{\theta}{2}\right)\lvert 00\rangle \;+\; \sin\!\left(\tfrac{\theta}{2}\right)\lvert 11\rangle
$$

tako da se ishodi merenja svode na:

$$
P_{\text{model}}(00) = \cos^2\!\left(\tfrac{\theta}{2}\right), \qquad
P_{\text{model}}(11) = \sin^2\!\left(\tfrac{\theta}{2}\right).
$$

Ovaj model može da predstavi raspodelu verovatnoća merenja rezultata $\{00,11\}$.

## Ključne funkcije

- **`vqc(theta, plot=False)`**  
  Generiše varijaciono kvantno kolo, izvršava ga na simulatoru i vraća dobijenu raspodelu verovatnoća.  

- **Funkcije odstupanja (loss)**  
  Poredi modeliranu raspodela $p_{\text{model}}$ sa ciljnom raspodelom $p_{\text{target}}$.  

  - `l1_loss(p_model, t_dist)`  
    Računa $\ell_1$ (Manhetn) rastojanje:  
    $$
    L_1(p_{\text{model}}, p_{\text{target}}) \;=\; 
    \sum_{x} \big|\, p_{\text{model}}(x) - p_{\text{target}}(x)\,\big|
    $$
  
  - `l2_loss(p_model, t_dist)`  
    Računa $\ell_2$ (Euklidsko) rastojanje:  
    $$
    L_2(p_{\text{model}}, p_{\text{target}}) \;=\;
    \sqrt{\;\sum_{x} \big( p_{\text{model}}(x) - p_{\text{target}}(x) \big)^2}
    $$

- **`basic_optimizer(loss_fn, t_dist, step_size, max_iter)`**  
  Izvršava jednostavnu optimizacionu petlju:  
  - Počinje od $\theta = 0$.  
  - Postepeno povećava $\theta$.  
  - Procenjuje raspodelu i odstupanje u odnosu na ciljnu raspodelu.  
  - Zaustavlja se kada odstupanje počne da raste.  

---

## Zadatak

1. Pokrenuti program i optimizovati $\theta$ za zahtevanu ciljnu raspodelu:  
   `t_dist = {'00': 0.3, '11': 0.7}`  

2. Posmatrati kako optimizator podešava $\theta$ da minimizuje odstupanje.  
3. Zameniti `t_dist` drugim raspodelama nad $\{00,11\}$ (npr. `{'00': 0.5, '11': 0.5}`) i uporediti rezultate.  
4. Probati i `l1_loss` i `l2_loss` da bi se videlo kako se njihove metrike razlikuju.  

## Očekivani rezultat

- Graf koji prikazuje kako se odstupanje smanjuje tokom optimizacije.  
- Dijagram kola za obučeni QBM.  
- Dvostruki stubični dijagram $p_{\text{model}}$ i $t_{\text{dist}}$ za vizuelnu proveru poklapanja.  
- Ispis optimizovanog parametra i broja iteracija.  

## Eksperimenti

- Proširiti kolo tako da može da pokrije verovatnoće stanja $\lvert 01\rangle$ i $\lvert 10\rangle$.  
- Iz Pajton biblioteke primeniti gradijentnu metodu ažuriranja $\theta$.  
- Istražiti kako broj izvršavanja programa (shots) utiče na naučenu vrednost $\theta$ i konačno odstupanje.


In [ ]:
from IPython.display import display

from qiskit import QuantumCircuit
from qiskit_aer import AerSimulator
from qiskit.visualization import plot_distribution
from qiskit.visualization import circuit_drawer
import matplotlib.pyplot as plt
import numpy as np


def vqc(theta, plot=False):
    """
    Creates and runs a simple 2-qubit variational quantum circuit.

    Applies an RY rotation by angle `theta` on qubit 0 followed by a CNOT gate to entangle qubit 0 and 1.
    The circuit is measured, optionally plotted, and executed on a simulator.

    Args:
        theta (float): Rotation angle for the RY gate on qubit 0.
        plot (bool): If True, displays the circuit diagram.

    Returns:
        dict: Probability distribution of measurement outcomes (bitstrings).
    """
    qc = QuantumCircuit(2,2)
    
    # set state       
    qc.ry(theta, 0)
    qc.cx(0, 1)

    qc.barrier()
    qc.measure(range(2), range(2))
    if plot:
        display(circuit_drawer(qc, output="mpl"))
    
    # -- run the program on the simulator --
    simulator = AerSimulator()
    result = simulator.run(qc, shots=1000).result()
    counts = result.get_counts(qc)

    # convert counts to probability distribution
    total = sum(counts.values())
    p_model = {k: v / total for k, v in counts.items()}
    
    return p_model

def l1_loss(p_model, t_dist):
    """
    Computes the L1 loss (Manhattan distance) between two probability distributions.

    The L1 loss is the sum of absolute differences between corresponding probabilities 
    in the model and target distributions. Missing keys are treated as zero.

    Args:
        p_model (dict): The predicted/model probability distribution.
        t_dist (dict): The target/reference probability distribution.

    Returns:
        float: Total L1 loss between the two distributions.
    """
    keys = set(p_model.keys()).union(t_dist.keys())
    loss = 0.0
    for k in keys:
        model_prob = p_model.get(k, 0.0)    # use 0.0 if key not found
        target_prob = t_dist.get(k, 0.0)    # use 0.0 if key not found
        diff = abs(model_prob - target_prob)
        loss += diff
    return loss

def l2_loss(p_model, t_dist):
    """
    Computes the L2 loss (Euclidean distance) between two probability distributions.

    The L2 loss is the square root of the sum of squared differences between corresponding 
    probabilities in the model and target distributions. Missing keys are treated as zero.

    Args:
        p_model (dict): The predicted/model probability distribution.
        t_dist (dict): The target/reference probability distribution.

    Returns:
        float: Total L2 loss between the two distributions.
    """

    keys = set(p_model.keys()).union(t_dist.keys())
    loss = 0.0
    for k in keys:
        model_prob = p_model.get(k, 0.0)    # use 0.0 if key not found
        target_prob = t_dist.get(k, 0.0)    # use 0.0 if key not found
        diff = model_prob - target_prob
        loss += diff ** 2
    return loss ** 0.5   # square root for L2


def basic_optimizer(loss_fn, t_dist, step_size=0.1, max_iter=100):
    """
    Optimizes the rotation angle theta to minimize the loss 
    between the VQC output distribution and a target distribution.

    The function incrementally increases theta, evaluates the VQC output, 
    and computes the loss against the target. Optimization stops when 
    the loss increases, indicating a minimum has been passed.

    Args:
        loss_fn: loss function name
        t_dist (dict): Target probability distribution.
        step_size (float): Increment for theta in each iteration.
        max_iter (int): Maximum number of optimization steps.

    Returns:
        tuple: Lists of theta values and corresponding losses.
    """
    theta = 0.0
    losses = []
    thetas = []

    prev_loss = None

    for i in range(max_iter):
        # get model distribution from VQC at current theta
        p_model = vqc(theta)

        # compute L1 loss
        loss = loss_fn(p_model, t_dist)

        # check if loss starts increasing
        if prev_loss is not None and loss > prev_loss:
            losses.pop() # discard the last guess since it performed worse 
            thetas.pop()
            break
            
        # store for graphing
        thetas.append(theta)
        losses.append(loss)

        prev_loss = loss
        theta += step_size

    return thetas, losses, i

# ------------------------------------------------
#                 main program
# ------------------------------------------------
# target distribution
t_dist = {'00': 0.3, '11': 0.7} 
thetas, losses, iter = basic_optimizer(l2_loss, t_dist)

# plot results
plt.plot(thetas, losses, marker='o')
plt.xlabel("θ")
plt.ylabel("L2 Loss")
plt.title("Loss vs θ")
plt.grid(True)
plt.show()

# display circuit and results with final paramter
model_dist = vqc(thetas[-1], True)
display(
    plot_distribution(
        [t_dist, model_dist],
        title="Target vs Model Distribution",
        legend=["Target (requested)", "Model"]
    )
)
print(f"Optimization finished after {iter} iterations with θ = {thetas[-1]}")
